Импорт пакетов

In [67]:
from numpy import asarray
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from matplotlib import pyplot
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd
from datetime import date
import numpy as np
import datetime

Импорт данных

In [68]:
series = pd.read_csv('D:\data12.csv', header=0, index_col=0)
data = pd.DataFrame()
data = pd.concat([data,series['Значение']],axis = 1)

Определение модели Random forest

In [69]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
# spl
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]
def random_forest_forecast(train, testX):
    # transform list into array
    train = asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    isBoost = False
    if isBoost==False :
        model = RandomForestRegressor(n_estimators=1000)
        model.fit(trainX, trainy)
    else:    
        model= GradientBoostingRegressor(n_estimators=1000).fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict([testX])
    return yhat[0]
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

Получение предсказаний модели Random forest

In [70]:
import time
t0 = time.time()
n_in=12
values = data.values

data = series_to_supervised(values, n_in=n_in,n_out=1)


mae, y, yhat = walk_forward_validation(data, n_in)
t = datetime.timedelta(seconds=time.time()-t0)
print(f'Time: {t}')

>expected=1715408.0, predicted=1608735.8
>expected=2149000.0, predicted=1692891.2
>expected=1778000.0, predicted=1525150.5
>expected=2011000.0, predicted=1731378.8
>expected=2219000.0, predicted=2051511.3
>expected=2115000.0, predicted=2132285.4
>expected=2275000.0, predicted=2085339.1
>expected=2496000.0, predicted=2178088.1
>expected=2513000.0, predicted=2182050.3
>expected=2705000.0, predicted=2226094.3
>expected=2713000.0, predicted=2337319.4
>expected=2083000.0, predicted=1947505.7
Time: 0:00:29.756461
